# Allocating Qubits on QPU Devices

In [1]:
# Use Braket SDK Cost Tracking to estimate the cost to run this example
from braket.tracking import Tracker

t = Tracker().start()

This notebook demonstrates how you can specify explicitly which qubits to use when you run a quantum circuit on QPU devices from Rigetti.

When you submit a circuit for execution on a QPU, Amazon Braket performs a series of compilation steps: it maps the _abstract qubits_ in your circuit to _physical qubits_ in the device; it synthesizes gates into the native gate set of the device; it optimizes the circuit to reduce the number of gates; and finally, it translates the gates into executable pulses.

This section shows how the first step, called qubit allocation, works for the Rigetti Ankaa-2 device.

In [2]:
# general imports
import random

import numpy as np

from braket.aws import AwsDevice
from braket.circuits import Circuit
from braket.devices import Devices

## Automatic qubit allocation

Qubit allocation for Rigetti devices on Amazon Braket utilizes [the Quil Compilers](https://pyquil-docs.rigetti.com/en/latest/compiler.html#the-quil-compiler)'s _rewiring_ strategies. By default, when you submit a circuit on Amazon Braket to a Rigetti device, the circuit is rewired according to the [PARTIAL](https://pyquil-docs.rigetti.com/en/latest/compiler.html#partial) rewiring strategy. Specifically, the compiler starts with an empty mapping from logical to physical qubits. Taking into account the latest calibration data of the device, the compiler fills in the mapping with the goal, sequentially, to maximize the overall fidelity of the circuit.

The example that follows shows how to create a GHZ state on qubits that are not physically connected. After the quantum task is completed, you can obtain a list of the actual gates executed on the device, by viewing the result metadata.

First, instantiate the Rigetti Ankaa-2 device and retrieve its connectivity graph, which shows the qubits that are directly connected on the chip.

In [3]:
device = AwsDevice(Devices.Rigetti.Ankaa2)

connectivity_graph = device.properties.paradigm.connectivity.connectivityGraph
print(f"the connectivity of {device.name} is: {connectivity_graph}")

the connectivity of Ankaa-2 is: {'0': ['1', '7'], '1': ['0', '2', '8'], '2': ['1', '3', '9'], '3': ['2', '4', '10'], '4': ['3', '5', '11'], '5': ['4', '6', '12'], '6': ['5', '13'], '7': ['0', '8', '14'], '8': ['1', '7', '9', '15'], '9': ['2', '8', '10', '16'], '10': ['3', '9', '11', '17'], '11': ['4', '10', '12', '18'], '12': ['5', '11', '13', '19'], '13': ['6', '12', '20'], '14': ['7', '15', '21'], '15': ['8', '14', '22'], '16': ['9', '17', '23'], '17': ['10', '16', '18', '24'], '18': ['11', '17', '19', '25'], '19': ['12', '18', '20', '26'], '20': ['13', '19', '27'], '21': ['14', '22', '28'], '22': ['15', '21', '23', '29'], '23': ['16', '22', '24', '30'], '24': ['17', '23', '25', '31'], '25': ['18', '24', '26', '32'], '26': ['19', '25', '33'], '27': ['20', '34'], '28': ['21', '29', '35'], '29': ['22', '28', '30', '36'], '30': ['23', '29', '31', '37'], '31': ['24', '30', '32', '38'], '32': ['25', '31', '33', '39'], '33': ['26', '32', '34', '40'], '34': ['27', '33', '41'], '35': ['28', 

Next, create a GHZ circuit with three qubits 0, 2, 4, and run it on the Ankaa 2 device. Notice that none of these qubits are connected on the Ankaa 2 connectivity graph.

In [4]:
# create a GHZ state with non-neighboring qubits
circuit = Circuit()
circuit.h(0).cnot(0, 2).cnot(0, 4)
print(circuit)

rigetti_rewiring = device.run(circuit, shots=10)

T  : │  0  │  1  │  2  │
      ┌───┐             
q0 : ─┤ H ├───●─────●───
      └───┘   │     │   
            ┌─┴─┐   │   
q2 : ───────┤ X ├───┼───
            └───┘   │   
                  ┌─┴─┐ 
q4 : ─────────────┤ X ├─
                  └───┘ 
T  : │  0  │  1  │  2  │


In [5]:
print("Status of quantum task:", rigetti_rewiring.state())

Status of quantum task: QUEUED


To verify the final qubit allocation, retrieve the compiled program that was executed:

In [6]:
result = rigetti_rewiring.result()
counts = result.measurement_counts
print("Measurement counts:", counts)
print("The compiled circuit is:\n", result.additional_metadata.rigettiMetadata.compiledProgram)

Measurement counts: Counter({'111': 6, '000': 3, '101': 1})
The compiled circuit is:
 PRAGMA INITIAL_REWIRING "NAIVE"
DECLARE ro BIT[3]
PRAGMA PRESERVE_BLOCK
RX(1.5707963267948966) 5
RZ(3.141592653589793) 5
ISWAP 5 12
RZ(1.5707963267948966) 5
RX(1.5707963267948966) 5
RZ(4.71238898038469) 5
ISWAP 5 12
RZ(1.5707963267948966) 5
RZ(3.141592653589793) 12
ISWAP 5 6
RX(1.5707963267948966) 12
RZ(1.5707963267948966) 5
RX(1.5707963267948966) 5
RZ(4.71238898038469) 5
ISWAP 5 6
RZ(3.141592653589793) 6
RX(1.5707963267948966) 6
PRAGMA END_PRESERVE_BLOCK
MEASURE 12 ro[1]
MEASURE 5 ro[0]
MEASURE 6 ro[2]


Notice that the PARTIAL rewiring was applied. The qubits 0, 2, 4 in the original circuit were mapped to three other qubits in the Rigetti device, and the gates were compiled into native gates.

## User-defined qubit allocation

In Amazon Braket, you can choose to prescribe a qubit mapping manually, and prevent further rewiring for Rigetti devices. To enable manual mapping, set `disable_qubit_rewiring=True` when submitting the quantum task to run.

If all the gates in the circuit satisfy the topological constraints of the device, Amazon Braket maps abstract qubit $i$ in the circuit to the physical qubit $i$ in the device, and maps qubit pair $(i, j)$ to the connection $(i, j)$ in the device. On the other hand, Amazon Braket raises an exception if a specified qubit or qubit pair do not exist in the device connectivity graph.

In [7]:
# create a random state with neighboring qubits
q1 = random.choice(list(connectivity_graph))
q2 = int(connectivity_graph[q1][0])
q1 = int(q1)

circuit = Circuit()
circuit.rz(0, np.pi / 2).cnot(q1, q2).x(7)
print(circuit)
rigetti_task = device.run(circuit, shots=10, disable_qubit_rewiring=True)

T  : │     0      │  1  │
      ┌──────────┐       
q0 : ─┤ Rz(1.57) ├───●───
      └──────────┘   │   
                   ┌─┴─┐ 
q1 : ──────────────┤ X ├─
                   └───┘ 
         ┌───┐           
q7 : ────┤ X ├───────────
         └───┘           
T  : │     0      │  1  │


In [8]:
print("Status of quantum task:", rigetti_task.state())

Status of quantum task: QUEUED


In [9]:
result = rigetti_task.result()
counts = result.measurement_counts
print("Measurement counts:", counts)
print("The compiled circuit is:\n", result.additional_metadata.rigettiMetadata.compiledProgram)

Measurement counts: Counter({'001': 7, '000': 2, '011': 1})
The compiled circuit is:
 PRAGMA INITIAL_REWIRING "NAIVE"
DECLARE ro BIT[3]
PRAGMA PRESERVE_BLOCK
RX(1.5707963267948966) 0
RX(1.5707963267948966) 7
RZ(4.71238898038469) 0
RX(1.5707963267948966) 7
ISWAP 0 1
RZ(3.141592653589793) 1
RX(1.5707963267948966) 1
PRAGMA END_PRESERVE_BLOCK
MEASURE 7 ro[2]
MEASURE 0 ro[0]
MEASURE 1 ro[1]


The qubits in the original circuit followed a one-to-one mapping to the physical qubits in the device. Other compilation steps, such as gate synthesis and circuit optimization, are still performed. These steps allow the circuit to run successfully and improve the overall fidelity.

### Using the qubits with the highest two-qubit gate fidelity

Additionally, the device properties include calibration data, which you can use to find the qubits and qubit pairs with the highest fidelities for particular gates.

The following function finds the qubit pair that has the highest two-qubit fidelity of an input gate, which can be any of the gates native to the Rigetti device. First, you can access the native gates as follows:

In [10]:
native_gates = device.properties.paradigm.nativeGateSet
gates_uppercase = [gate.upper() for gate in native_gates]
gates_uppercase

['RX', 'RZ', 'CZ', 'ISWAP']

In [11]:
def find_qubit_pair(gate):
    "Function to find the qubit pair that has the highest gate fidelity of a particular gate"

    # check whether the input gate is a string
    if not isinstance(gate, str):
        raise ValueError("The input gate must be a string type.")

    # check whether the input gate is a native gate
    gate_list = gates_uppercase
    if gate not in gate_list:
        raise ValueError(f"The input gate must be one of {gates_uppercase}.")

    # load all calibration data from device.properties
    calibration_2Q = device.properties.provider.specs["2Q"]
    highest_fidelity = 0

    # iterate through all calibration data to find the highest fidelity
    for pair in calibration_2Q.keys():
        # if the particular gate type is supported by the qubit pair
        if ("f" + gate) in calibration_2Q[pair].keys():
            if calibration_2Q[pair]["f" + gate] > highest_fidelity:
                # update the highest_fidelity and the best_pair
                highest_fidelity = calibration_2Q[pair]["f" + gate]
                best_pair = pair

    # generate the two qubits as integers
    q1 = best_pair[0]
    i = 1
    while best_pair[i] != "-":
        q1 += best_pair[i]
        i += 1

    q1 = int(q1)
    q2 = int(best_pair[i + 1 :])

    return q1, q2, highest_fidelity

The example in the following code applies a native two-qubit gate on the qubit pair that has the highest fidelity of that gate. 

In [12]:
# the gate must be a native gate
gate = "ISWAP"
# find the qubit pair with the highest gate fidelity
q1, q2, highest_fidelity = find_qubit_pair(gate)
print("The highest fidelity for " + gate + " gate is:", highest_fidelity)
print(f"And the corresponding qubit pair is: qubit {q1} and qubit {q2}")

# create a circuit with the gate applied to the discovered qubit pair.
# note that CPHASE in Rigetti corresponds to cphaseshift in Braket
circuit = Circuit()
circuit.cz(q1, q2)
print(circuit)
rigetti_task = device.run(circuit, shots=1000, disable_qubit_rewiring=True)

The highest fidelity for ISWAP gate is: 0.99312484820244
And the corresponding qubit pair is: qubit 73 and qubit 74
T   : │  0  │
             
q73 : ───●───
         │   
       ┌─┴─┐ 
q74 : ─┤ Z ├─
       └───┘ 
T   : │  0  │


In [13]:
print("Status of quantum task:", rigetti_task.state())

Status of quantum task: QUEUED


The qubits in the original circuit followed a one-to-one mapping to the physical qubits in the device. Since only native gates were used, the actual gates executed are the same as the gates in the original circuit.

<div class="alert alert-block alert-info">
<b>Note:</b> The IonQ device does not support manual allocation. For circuits submitted to the IonQ device, qubits are allocated automatically.
</div>

In [14]:
print("Quantum Task Summary")
print(t.quantum_tasks_statistics())
print(
    "Note: Charges shown are estimates based on your Amazon Braket simulator and quantum processing unit (QPU) task usage. Estimated charges shown may differ from your actual charges. Estimated charges do not factor in any discounts or credits, and you may experience additional charges based on your use of other services such as Amazon Elastic Compute Cloud (Amazon EC2)."
)
print(
    f"Estimated cost to run this example: {t.qpu_tasks_cost() + t.simulator_tasks_cost():.2f} USD"
)

Quantum Task Summary
{<_Rigetti.Ankaa2: 'arn:aws:braket:us-west-1::device/qpu/rigetti/Ankaa-2'>: {'shots': 1020, 'tasks': {'COMPLETED': 2, 'QUEUED': 1}}}
Note: Charges shown are estimates based on your Amazon Braket simulator and quantum processing unit (QPU) task usage. Estimated charges shown may differ from your actual charges. Estimated charges do not factor in any discounts or credits, and you may experience additional charges based on your use of other services such as Amazon Elastic Compute Cloud (Amazon EC2).
Estimated cost to run this example: 1.82 USD
